In [ ]:
# 需要使用高RAM的機器，免費版的colab目前跑不動

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
def read_vectors_from_file(file_path): #讀song_id和對應的vector，並處理成一個字典對照表 一個所有vectors 的array
    vectors = []
    dic = {}
    skip_first = True
    with open(file_path, 'r') as file:
        for line in file:
            if skip_first:
                skip_first = False
                continue
            # 移除空白字符，然後以逗號分隔
            vector_strings = line.strip().split(' ')

            if len(vector_strings[0]) < 10:  # not hash code
                continue
            if vector_strings[0][0:3] in ["com", "gen", "lyr", "prd", "ttl"]:
                continue

            # 將字符串轉換為浮點數
            vector = [float(num) / 10 for num in vector_strings[1:]]
            dic[vector_strings[0]] = vector
            # 將向量添加到列表中
            vectors.append(vector)

    return dic, np.array(vectors)

def read_vectors_from_csv(file_path):
    df = pd.read_csv(file_path)
    return df

In [ ]:
dic, vectors = read_vectors_from_file("/content/drive/MyDrive/datagame/hpe_all_ts.txt")
# 將song_id和歌手、作詞者、作曲者、發行人之類的資訊合併起來，用smore跑hpe，把song_id轉換成64個數字

In [ ]:
dic_k = list(dic.keys())
dic_v = list(dic.values())
reverse_dic = {tuple(v): k for k, v in dic.items()}

In [ ]:
train_source_df = read_vectors_from_csv("/content/drive/MyDrive/datagame/label_train_source.csv")
train_source_df.sort_values(by=['session_id', "listening_order"], inplace=True)
train_source_df.reset_index(inplace=True, drop=True)
train_target_df = read_vectors_from_csv("/content/drive/MyDrive/datagame/label_train_target.csv")
train_target_df.sort_values(by=['session_id', "listening_order"], inplace=True)
train_target_df.reset_index(inplace=True, drop=True)

In [ ]:
X = np.zeros( (len(train_source_df) // 20, 20, 64) )
y = np.zeros( (len(train_target_df) // 5,  64 * 5) )

for i in range(len(train_source_df)):
    X[ i // 20 ][ i % 20 ] = dic[ train_source_df.loc[i, "song_id"] ]

for i in range(len(train_target_df)):
    y[ i // 5 ][ (i % 5) * 64 : (i % 5 + 1) * 64 ] = dic[ train_target_df.loc[i, "song_id"] ]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)
del(X)
del(y)

In [ ]:
modelcheckpoint = keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/datagame/LSTM_simple.h5",
    monitor="val_loss",verbose=0,save_best_only=True,
    save_weights_only=True,mode="min",)

earlystop = keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,verbose=0,mode="min")

reduces = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                        factor=0.1,
                        patience=5,
                        verbose=1,
                        mode='min',
                        cooldown=1 )
callbacks = [modelcheckpoint, reduces, earlystop]

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

act = 'tanh'

model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(20, 64)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation=act),

    # Bidirectional LSTM layers
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, activation=act)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True, activation=act)),

    # Final LSTM layer (unidirectional)
    tf.keras.layers.LSTM(512, activation=act),
    tf.keras.layers.Dense(320, activation=act)
])

optimizer = tf.keras.optimizers.Lion(learning_rate = 0.0001)
model.compile(loss="mae", optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 20, 64)            256       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 20, 128)           8320      
                                                                 
 bidirectional (Bidirection  (None, 20, 512)           788480    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 20, 1024)          4198400   
 onal)                                                           
                                                                 
 lstm_2 (LSTM)               (None, 512)               3147776   
                                                        

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=100, callbacks=callbacks, batch_size = 1024,
          validation_data=(X_test, y_test))

Epoch 1/100
503/503 [==============================] - 66s 114ms/step - loss: 0.0215 - val_loss: 0.0207 - lr: 1.0000e-04
Epoch 2/100
503/503 [==============================] - 54s 108ms/step - loss: 0.0193 - val_loss: 0.0191 - lr: 1.0000e-04
Epoch 3/100
503/503 [==============================] - 54s 108ms/step - loss: 0.0191 - val_loss: 0.0190 - lr: 1.0000e-04
Epoch 4/100
503/503 [==============================] - 54s 108ms/step - loss: 0.0189 - val_loss: 0.0189 - lr: 1.0000e-04
Epoch 5/100
503/503 [==============================] - 54s 108ms/step - loss: 0.0189 - val_loss: 0.0189 - lr: 1.0000e-04
Epoch 6/100
503/503 [==============================] - 54s 108ms/step - loss: 0.0188 - val_loss: 0.0188 - lr: 1.0000e-04
Epoch 7/100
503/503 [==============================] - 54s 108ms/step - loss: 0.0188 - val_loss: 0.0188 - lr: 1.0000e-04
Epoch 8/100
503/503 [==============================] - 55s 109ms/step - loss: 0.0187 - val_loss: 0.0187 - lr: 1.0000e-04
Epoch 9/100
503/503 [===========

KeyboardInterrupt: ignored

In [ ]:
# Make predictions on test data
predicted_target = model.predict(X_test)

716/716 [==============================] - 4s 4ms/step


In [ ]:
print(sum(tf.keras.losses.mean_squared_error(y_test, predicted_target) ** 0.5) / len(X_test))

tf.Tensor(0.048978284, shape=(), dtype=float32)


# test

In [ ]:
# load the model
model.load_weights('/content/drive/MyDrive/datagame/LSTM_simple.h5')

# Initialize the decoder input with zeros
test_df = read_vectors_from_csv("/content/drive/MyDrive/datagame/label_test_source.csv")
test_df.sort_values(by=['session_id', "listening_order"], inplace=True)
test_df.reset_index(inplace=True, drop=True)

In [ ]:
X_result = np.zeros( (len(test_df) // 20, 20, 64) )
test_session_id = []

for i in range(len(test_df)):
    X_result[ i // 20 ][ i % 20 ] = dic[ test_df.loc[i, "song_id"] ]
    if i % 20 == 0:
        test_session_id.append(test_df.loc[i, "session_id"])

In [ ]:
import cupy as cp
already_used = []
def find_closest_vector(target_vector):
    # 計算與目標向量的距離
    distances_squared = cp.sum((vectors - target_vector) ** 2, axis=1)
    # 找出距離最近的 k 個向量的索引，並跳過已經出現過的vector
    closest_index = cp.argmin(distances_squared)
    while closest_index in already_used:
      distances_squared[closest_index] = 9999999
      closest_index = cp.argmin(distances_squared)

    # 返回最近的 k 個向量
    already_used.append(closest_index)
    return vectors[closest_index]

def find_key_by_value(value_to_find):
    value_to_find = np.array(value_to_find.get())
    value_tuple = tuple(value_to_find)
    return reverse_dic.get(value_tuple, None)

In [ ]:
# Predict the next vector for each sequence in the test set
final_y = model.predict(X_result)

4471/4471 [==============================] - 25s 5ms/step


In [ ]:
# change vector to song id
final_result = np.zeros(( len(test_session_id), 5)).astype(str)
vectors = cp.array(vectors)
final_y = cp.array(final_y)
for i in range(len(test_session_id)):
    if(i % 5000 == 0):
      print(f"{ str( round(i*100 / len(test_session_id), 2) ) }" + "%")
    already_used.clear()
    for j in range(5):
        final_result[i, j] = find_key_by_value( find_closest_vector( final_y[i, j*64 : (j+1)*64] ) )

0.0%
3.49%
6.99%
10.48%
13.98%
17.47%
20.97%
24.46%
27.96%
31.45%
34.95%
38.44%
41.94%
45.43%
48.93%
52.42%
55.92%
59.41%
62.91%
66.4%
69.9%
73.39%
76.89%
80.38%
83.88%
87.37%
90.87%
94.36%
97.86%


In [ ]:
final_result[:5, :2]

array([['bf96b92cf2afe619e4307bc972f8f0df',
        '4f1f6db6a858baec3779db114eb6b985'],
       ['b897d90f0c8989a40051950606edc798',
        '32142bf5411ff50d0d6a795db5ff754c'],
       ['6accb2b1efc3ef2e50c1b893ed84bc97',
        '25dbafeee3fae51a805bdc73ab5051b1'],
       ['822560a5231b62186417aa98da173d16',
        '89840d06c35f2d25083be9e742d0d577'],
       ['06de4491a1d215da9e7d69acae71f9b8',
        '0e373df3c954f80a912a4c8ec05c2f44']], dtype='<U32')

In [ ]:
import pandas as pd
final_df = pd.DataFrame(columns=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'])
final_df["session_id"] = test_session_id
final_df["top1"] = final_result[:, 0]
final_df["top2"] = final_result[:, 1]
final_df["top3"] = final_result[:, 2]
final_df["top4"] = final_result[:, 3]
final_df["top5"] = final_result[:, 4]

final_df.to_csv("/content/drive/MyDrive/datagame/LSTM_result.csv", index=False)